In [5]:
import dspy
from helper import lm
dspy.settings.configure(lm=lm())

In [6]:
class TextToSQL(dspy.Signature):
    """Transform a natural language query into a SQL query."""

    sql_prompt = dspy.InputField(desc="Natural language query")
    sql_context = dspy.InputField(desc="Context for the query")
    sql_output = dspy.OutputField(desc="SQL query")

In [24]:
predict = dspy.ChainOfThought(TextToSQL)

result = predict(
    sql_prompt = """Write a query to fetch values in table test_a that are not in test_b""",
    sql_context = """Table test_a schema is test_a(id INT).
    Table test_b schema is test_b(id INT)"""
)

for key, value in result.items():
    print(f"\n{key.upper()}\n")
    print(value)

        


RATIONALE

produce the sql_output. We need to select the `id` values from `test_a` that do not exist in `test_b`. This can be achieved using a `LEFT JOIN` or a `NOT IN` clause. For simplicity, we will use the `NOT IN` clause to filter out the `id` values from `test_a` that are present in `test_b`.

SQL_OUTPUT

```sql
SELECT id
FROM test_a
WHERE id NOT IN (SELECT id FROM test_b);
```


### Try relation table

In [28]:
predict = dspy.ChainOfThought(TextToSQL)

result = predict(
    sql_prompt = """Get list of employees who took a training lesson more than once in the same day, group by user and training lesson, 
    each ordered from the most recent lesson date to oldest date""",
    sql_context = """Employee table schema is, (emp_id INT, emp_name VARCHAR(500))
      Training detail table schema is,() emp_training_id INT, emp_id INT, training_id INT, training_date DATE)"""
)

for key, value in result.items():
    print(f"\n{key.upper()}\n")
    print(value)


RATIONALE

produce the sql_output. We need to:

1. Identify the employees who took the same training lesson more than once on the same day.
2. Group the results by employee and training lesson.
3. Order the results by the most recent training date to the oldest date.

To achieve this, we will:
1. Use a subquery to count the occurrences of each training lesson taken by each employee on each day.
2. Filter the subquery to include only those records where the count is greater than one.
3. Join the filtered results back with the original training detail table to get the complete details.
4. Group by employee and training lesson.
5. Order the results by training date in descending order.

Here is the SQL query:

Sql

SQL_OUTPUT

```sql
SELECT 
    e.emp_id, 
    e.emp_name, 
    t.training_id, 
    t.training_date
FROM 
    Employee e
JOIN 
    TrainingDetail t ON e.emp_id = t.emp_id
JOIN (
    SELECT 
        emp_id, 
        training_id, 
        training_date, 
        COUNT(*) as train